In [45]:
import pandas as pd
import time
import random
from abc import ABC, abstractmethod
from typing import List, Dict, Optional, Generator
from nba_api.stats.endpoints import leaguegamefinder, playbyplayv2, playbyplay, playbyplayv3, boxscoreadvancedv3, teamgamelog
from nba_api.stats.static import teams
import nba_api.live

In [7]:
def get_season_games(self, season: str = '2023-24') -> List[str]:
        """
        Fetches all game IDs for a given season.
        Format for season is 'YYYY-YY' (e.g., '2023-24').
        """
        print(f"Fetching games for season {season}...")
        # league_id_nullable='00' is NBA
        gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season, league_id_nullable='00', timeout=30)
        games = gamefinder.get_data_frames()[0]
        return games['GAME_ID'].unique().tolist()

In [8]:
get_season_games(12)

Fetching games for season 2023-24...


['0042300405',
 '0042300404',
 '0042300403',
 '0042300402',
 '0042300401',
 '0042300315',
 '0042300314',
 '0042300304',
 '0042300313',
 '0042300303',
 '0042300312',
 '0042300302',
 '0042300311',
 '0042300301',
 '0042300237',
 '0042300217',
 '0042300226',
 '0042300216',
 '0042300236',
 '0042300225',
 '0042300205',
 '0042300235',
 '0042300215',
 '0042300224',
 '0042300204',
 '0042300234',
 '0042300214',
 '0042300223',
 '0042300203',
 '0042300213',
 '0042300233',
 '0042300222',
 '0042300202',
 '0042300212',
 '0042300201',
 '0042300221',
 '0042300232',
 '0042300211',
 '0042300137',
 '0042300231',
 '0042300176',
 '0042300136',
 '0042300126',
 '0042300116',
 '0042300105',
 '0042300175',
 '0042300125',
 '0042300115',
 '0042300135',
 '0042300104',
 '0042300144',
 '0042300155',
 '0042300114',
 '0042300164',
 '0042300174',
 '0042300124',
 '0042300154',
 '0042300134',
 '0042300143',
 '0042300103',
 '0042300163',
 '0042300173',
 '0042300123',
 '0042300113',
 '0042300153',
 '0042300133',
 '00423001

In [12]:
pbp_endpoint = playbyplayv3.PlayByPlayV3(game_id="0042300313", timeout=30)

In [ ]:
def get_game_pbp(self, game_id: str) -> pd.DataFrame:
        """
        Fetches raw PBP data for a specific game and standardizes the schema.
        Includes retry logic and fallback to v1 endpoint.
        """
        print(f"Fetching PBP for game {game_id}...")

        try:
            pbp_endpoint = playbyplayv3.PlayByPlayV3(game_id=game_id, timeout=30)
            raw_df = pbp_endpoint.get_data_frames()[0]
        except Exception as e:
            print(f"V3 failed ({e})")
            return None
        return raw_df

In [16]:
get_game_pbp(12, "0042300215").columns

Fetching PBP for game 0042300215...


Index(['gameId', 'actionNumber', 'clock', 'period', 'teamId', 'teamTricode',
       'personId', 'playerName', 'playerNameI', 'xLegacy', 'yLegacy',
       'shotDistance', 'shotResult', 'isFieldGoal', 'scoreHome', 'scoreAway',
       'pointsTotal', 'location', 'description', 'actionType', 'subType',
       'videoAvailable', 'shotValue', 'actionId'],
      dtype='object')

In [42]:
try:
    endpoint = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id="0042300215", timeout=30)
    raw_df = endpoint.get_data_frames()[0]
except Exception as e:
    print(f" failed ({e})")

print(raw_df.columns)
raw_df.head()

Index(['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug',
       'personId', 'firstName', 'familyName', 'nameI', 'playerSlug',
       'position', 'comment', 'jerseyNum', 'minutes',
       'estimatedOffensiveRating', 'offensiveRating',
       'estimatedDefensiveRating', 'defensiveRating', 'estimatedNetRating',
       'netRating', 'assistPercentage', 'assistToTurnover', 'assistRatio',
       'offensiveReboundPercentage', 'defensiveReboundPercentage',
       'reboundPercentage', 'turnoverRatio', 'effectiveFieldGoalPercentage',
       'trueShootingPercentage', 'usagePercentage', 'estimatedUsagePercentage',
       'estimatedPace', 'pace', 'pacePer40', 'possessions', 'PIE'],
      dtype='object')


,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE
0,0042300215,1610612754,Indiana,Pacers,IND,pacers,1630174,Aaron,Nesmith,A. Nesmith,...,16.7,0.357,0.481,0.169,0.169,94.25,94.25,78.54,54.0,0.031
1,0042300215,1610612754,Indiana,Pacers,IND,pacers,1627783,Pascal,Siakam,P. Siakam,...,9.1,0.533,0.608,0.274,0.274,95.47,95.47,79.56,66.0,0.128
2,0042300215,1610612754,Indiana,Pacers,IND,pacers,1626167,Myles,Turner,M. Turner,...,22.2,0.667,0.667,0.258,0.258,93.44,93.44,77.87,58.0,0.112
3,0042300215,1610612754,Indiana,Pacers,IND,pacers,1629614,Andrew,Nembhard,A. Nembhard,...,11.8,0.500,0.506,0.232,0.232,92.06,92.06,76.72,52.0,0.061
4,0042300215,1610612754,Indiana,Pacers,IND,pacers,1630169,Tyrese,Haliburton,T. Haliburton,...,11.8,0.611,0.658,0.169,0.169,93.97,93.97,78.31,66.0,0.118


In [40]:
try:
    endpoint = teamgamelog.TeamGameLog(team_id=1610612754, timeout=30)
    raw_df = endpoint.get_data_frames()[0]
except Exception as e:
    print(f" failed ({e})")
print(raw_df.columns)
raw_df.head()

Index(['Team_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'W', 'L', 'W_PCT',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS'],
      dtype='object')


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612754,0022500048,"NOV 21, 2025",IND @ CLE,L,2,14,0.125,240,37,...,0.923,10,33,43,25,10,6,12,22,109
1,1610612754,0022500256,"NOV 19, 2025",IND vs. CHA,W,2,13,0.133,240,46,...,0.826,10,33,43,32,5,8,15,22,127
2,1610612754,0022500242,"NOV 17, 2025",IND @ DET,L,1,13,0.071,240,35,...,0.733,20,23,43,19,8,5,21,26,112
3,1610612754,0022500230,"NOV 15, 2025",IND vs. TOR,L,1,12,0.077,240,37,...,0.815,7,33,40,22,5,3,15,25,111
4,1610612754,0022500226,"NOV 13, 2025",IND @ PHX,L,1,11,0.083,240,32,...,0.931,16,25,41,20,4,3,12,22,98


In [41]:
len(raw_df)

16

In [49]:
from nba_api.live.nba.endpoints import scoreboard, boxscore
import pandas as pd
import time

# 1. Get Today's Scoreboard to find active games
board = scoreboard.ScoreBoard()
games = board.games.get_dict()

live_games = [g for g in games if g['gameStatus'] == 2] # Status 2 = In Progress

if not live_games:
    print("No games are currently live.")
else:
    # Pick the first live game
    target_game_id = live_games[0]['gameId']
    game_code = live_games[0]['gameCode']
    print(f"Fetching live stats for: {game_code} (ID: {target_game_id})")

    # 2. Get the Live Box Score
    box = boxscore.BoxScore(game_id=target_game_id)
    data = box.game.get_dict()

    # 3. Parse out a specific player's live stats (Example: Home Team)
    home_team_players = data['homeTeam']['players']
    
    # Convert to DataFrame for a "Box Score" view
    df = pd.DataFrame(home_team_players)
    
    # Clean up the view: Name, Points, Assists, Rebounds
    live_view = df[['name', 'points', 'assists', 'rebounds', 'plusMinus']]
    
    print(f"\n--- Live Box Score: {data['homeTeam']['teamName']} ---")
    print(live_view.sort_values(by='points', ascending=False).head(5).to_string(index=False))

No games are currently live.


In [52]:
print(games)

[{'gameId': '0022500283', 'gameCode': '20251124/DETIND', 'gameStatus': 1, 'gameStatusText': '7:00 pm ET', 'period': 0, 'gameClock': '', 'gameTimeUTC': '2025-11-25T00:00:00Z', 'gameEt': '2025-11-24T19:00:00Z', 'regulationPeriods': 4, 'ifNecessary': False, 'seriesGameNumber': '', 'gameLabel': '', 'gameSubLabel': '', 'seriesText': '', 'seriesConference': '', 'poRoundDesc': '', 'gameSubtype': '', 'isNeutral': False, 'homeTeam': {'teamId': 1610612754, 'teamName': 'Pacers', 'teamCity': 'Indiana', 'teamTricode': 'IND', 'wins': 2, 'losses': 14, 'score': 0, 'seed': None, 'inBonus': None, 'timeoutsRemaining': 0, 'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 0}, {'period': 2, 'periodType': 'REGULAR', 'score': 0}, {'period': 3, 'periodType': 'REGULAR', 'score': 0}, {'period': 4, 'periodType': 'REGULAR', 'score': 0}]}, 'awayTeam': {'teamId': 1610612765, 'teamName': 'Pistons', 'teamCity': 'Detroit', 'teamTricode': 'DET', 'wins': 14, 'losses': 2, 'score': 0, 'seed': None, 'inBonus': Non